In [36]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
# simple sequential chain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import os

In [37]:
os.environ["AZURE_OPENAI_API_KEY"] = "14cb551b1ce5493084f689735ac91281"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://caecoai0jiaoa01.openai.azure.com/"

In [38]:
with open('DWSD0612_completo_V3.txt', 'r') as file:
    data = file.read().strip()
    data = ' '.join(data.split())

In [39]:
final = []
divisions = ["ENVIRONMENT DIVISION", "DATA DIVISION","PROCEDURE DIVISION"]
data_split = data.split('DIVISION')[2:]
for i,row in enumerate(data_split):
    final.append(f'{divisions[i]} {row}')


In [41]:
model = AzureChatOpenAI(
    api_version="2023-05-15",
    azure_deployment="EY_FSO_ChatTest",
    temperature=0.7
)

In [42]:
code_analysis = """
    Given the following Cobol code, translate it into PySpark code:

    {code}
            
    Ensure that the PySpark code includes all the necessary processes.
    Return Only the Pyspark Code
    """


prompt = PromptTemplate(
    input_variables=["code"],
    template=code_analysis
)


chain = LLMChain(llm=model, prompt=prompt, output_key='explanation')



In [ ]:
responses = []
for code in tqdm(final):
    time.sleep(1)
    response = completion(prompt=f)
    try:
        responses.append(response.json())
    except:
        responses.append(response.text)

In [43]:
from tqdm import tqdm
responses = []
for code in tqdm(final):
    responses.append(chain.run(code=code))

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [02:17<00:00, 45.69s/it]


In [46]:
responses

['The COBOL code you\'ve provided is mainly setting up the environment and file assignments, which doesn\'t directly translate to PySpark code. However, I can show you how to load files (like the COBOL SELECT statement) into a PySpark environment. \n\n```python\nfrom pyspark.sql import SparkSession\n\n# Create a SparkSession\nspark = SparkSession.builder.appName(\'app\').getOrCreate()\n\n# Load the files into DataFrames\nARQPARM_df = spark.read.text("path_to_ARQPARM_file")\nARQSORT_df = spark.read.text("path_to_ARQSORT_file")\nDIMESTIP_df = spark.read.text("path_to_DIMESTIP_file")\nARQFTP_df = spark.read.text("path_to_ARQFTP_file")\n```\n\nThis code assumes that the files are in text format. If the files are in a different format (like CSV, Parquet, etc.), you would need to use the appropriate method (e.g., `read.csv("file_path")` or `read.parquet("file_path")`) instead of `read.text`.\n\nAlso, replace `"path_to_ARQPARM_file"`, `"path_to_ARQSORT_file"`, `"path_to_DIMESTIP_file"`, and `

In [53]:
import re

sparks = []
for resp in  responses:
    code = re.findall(r'```python\n(.*?)```', resp, re.DOTALL)[0]
    sparks.append(f'```python\n {code} ```') 
final_sparks = '\n\n\n '.join(sparks)

In [48]:
final_sparks

'```python\n from pyspark.sql import SparkSession\n\n# Create a SparkSession\nspark = SparkSession.builder.appName(\'app\').getOrCreate()\n\n# Load the files into DataFrames\nARQPARM_df = spark.read.text("path_to_ARQPARM_file")\nARQSORT_df = spark.read.text("path_to_ARQSORT_file")\nDIMESTIP_df = spark.read.text("path_to_DIMESTIP_file")\nARQFTP_df = spark.read.text("path_to_ARQFTP_file")\n ```\n\n\n ```python\n from pyspark.sql import SparkSession\n\n# Create a SparkSession\nspark = SparkSession.builder.appName("example").getOrCreate()\n\n# Read data from a CSV file\ndf = spark.read.csv("path_to_your_file.csv", header=True, inferSchema=True)\n\n# Perform some data manipulations\ndf = df.filter(df["column_name"] > 0)\n\n# Write the data back to a CSV file\ndf.write.csv("path_to_output_file.csv")\n ```\n\n\n ```python\n from pyspark.sql import SparkSession\nfrom pyspark.sql.functions import col\n\nspark = SparkSession.builder.getOrCreate()\n\n# Read input data\narqparm = spark.read.text("

In [52]:
with open("Outputs/Cobol2Spark_29.md", "w") as text_file:
        text_file.write('\n')
        text_file.write(final_sparks)
        text_file.write('\n')